In [1]:
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import tokenizers
import transformers
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

from parameters import *

import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
# Data Loading
def load_data():
    train = pd.read_csv(TRAIN_DIR)
    test = pd.read_csv(TEST_DIR)
    return train['full_text'].to_list(), train[train.columns[2:]].to_numpy(), test['text_id'].to_list(), test['full_text'].to_list()

In [ ]:
def MCRMSE(y_trues, y_preds):
    """_summary_

    Args:
        y_trues (_type_): _description_
        y_preds (_type_): _description_

    Returns:
        _type_: _description_
        _type_: _description_
    """
    scores = []
    idxes = y_trues.shape[1]
    for i in range(idxes):
        y_true = y_trues[:,i]
        y_pred = y_preds[:,i]
        score = mean_squared_error(y_true, y_pred, squared=False) # RMSE
        scores.append(score)
    mcrmse_score = np.mean(scores)
    return mcrmse_score, scores